In [1]:
# !pip install netCDF4
import netCDF4 as nc
import xarray as xr
import pandas as pd
import time
from ftplib import FTP
import os
import csv
from datetime import date, timedelta, datetime



In [2]:
# variable_dict = {
#     "aerosol_optical_thickness_over_water":"__GLOB_RESOLUTION_AVW-MODVIR_T865_DAY_00",
#     "aerosol_optical_thickness_over_water+land":"__GLOB_RESOLUTION_AVW-MODVIR_T550_DAY_00",
#     "cloud_fraction ":"__GLOB_RESOLUTION_AV-MODVIR_CF_DAY_00",
#     "depth_of_the_bottom_of_the_euphotic_layer":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZEU_DAY_00",
#     "secchi_disk_depth":"__GLOB_RESOLUTION_AVW-MODVIR_ZSD-DORON_DAY",
#     "heated_layer_depth":"__GLOB_RESOLUTION_AVW-MODVIR*_ZHL_DAY_00",
#     "particulate_backscattering_coefficient":"__GLOB_RESOLUTION_GSM-MODVIR_BBP_DAY_00",
#     "coloured_dissolved_detrital_organic_materials_absorption_coefficient":"__GLOB_RESOLUTION_GSM-MODVIR_CDM_DAY_00",
#     "diffuse_attenuation_coefficient":"__GLOB_RESOLUTION_AVW-MODVIR_KD490-LEE_DAY_00",
#     "relative_excess_of_radiance":"__GLOB_RESOLUTION_AVW-MODVIR_EL555_DAY_00",
#     "normalised_remote_sensing_reflectance":"__GLOB_RESOLUTION_AVW-MODVIR_NRRS490_DAY_00",
#     "normalised_fluorescence_line_height":"__GLOB_RESOLUTION_AV-MOD_NFLH_DAY_00",
#     "particulate_organic_carbon":"__GLOB_RESOLUTION_AVW-MODVIR_POC_DAY_00",
#     "particulate_inorganic_carbon":"__GLOB_RESOLUTION_AVW-MODVIR_PIC_DAY_00",
#     "inorganic_suspended_particulate_matter_concentration":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_SPM-OC5_DAY_00",
#     "photosynthetically_available_radiation":"__GLOB_RESOLUTION_AVW-MODVIR_PAR_DAY_00"
    
# }

# variable dictionary
reader = csv.DictReader(open('C:/Users/javi2/Documents/CD_aplicada_1/COBI/other/variable_dict.csv'))
variable_dict = {}

for row in reader:
    variable_dict[row['variable']] = row['file_format']

# # Ver qué pez con las variables de entorno
# directory_to = os.environ.get("PATH_SIMAR")

# credentials = [os.environ.get("GLOBCOLOUR_USER"), os.environ.get('GLOBCOLOUR_PWD')]
                                         
# Ver qué pez con las variables de entorno
directory_to = "C:\\Users\\javi2\\Documents\\CD_aplicada_1\\COBI\\data\\simar\\raw\\"

credentials = ['ftp_gc_JOrcazas_Leal', 'JOrcazas_Leal_6463']

In [3]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


In [4]:
def download_raw(variable, directory_to, resolution, credentials, variable_dict, year_from):

    user = credentials[0] 
    password = credentials[1] 

    directory_path = directory_to + variable
    if not os.path.isdir(directory_path):
        os.mkdir(directory_path)
    
    os.chdir(directory_path) #changes the active dir - this is where downloaded files will be saved to
    
    
    filematch = 'L3m_*'+variable_dict[variable].replace("RESOLUTION", resolution)+'.nc' # a match for any file in this case, can be changed or left for user to input
    
    not_loaded = []
    
    start_date = date(year_from, 1, 1)
    end_date = date(datetime.now().year, datetime.now().month, datetime.now().day-1)
    
    for single_date in daterange(start_date, end_date):
        
        year = single_date.strftime("%Y")
        month = single_date.strftime("%m")
        day = single_date.strftime("%d")
        try:                    
            download_dir ="GLOB/merged/day/"+year+"/"+month+"/"+day+"/" #dir i want to download files from, can be changed or left for user input
            ftp = FTP("ftp.hermes.acri.fr")
            ftp.login(user,password)

            ftp.cwd(download_dir)
            for filename in ftp.nlst(filematch): # Loop - looking for matching files
                if not os.path.exists(filename):
                    fhandle = open(filename, 'wb')
#                     print('Getting ' + filename) #for confort sake, shows the file that's being retrieved
                    ftp.retrbinary('RETR ' + filename, fhandle.write)
                    fhandle.close()

            ftp.quit()
        except Exception as e:
            not_loaded.append((year, month, day, e))
            print("error in variable", variable,":", e, year+month+day)

    return not_loaded

In [5]:
results = {}
for v in variable_dict.keys():
    results[v] = download_raw(v, directory_to, "100", credentials, variable_dict, 2017)

error in variable aerosol_optical_thickness_over_water : [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto 20180615
error in variable depth_of_the_bottom_of_the_euphotic_layer : 450 No files found 20170101
error in variable depth_of_the_bottom_of_the_euphotic_layer : 450 No files found 20170102
error in variable depth_of_the_bottom_of_the_euphotic_layer : 450 No files found 20170103
error in variable depth_of_the_bottom_of_the_euphotic_layer : 450 No files found 20170104
error in variable depth_of_the_bottom_of_the_euphotic_layer : 450 No files found 20170105
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170106
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170107
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170108
error in variable depth

error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170309
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170310
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170311
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170312
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170313
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170314
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170315
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170316
error in variabl

error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170515
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170516
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170517
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170518
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170519
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170520
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170521
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170522
error in variabl

error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170721
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170722
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170723
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170724
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170725
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170726
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170727
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170728
error in variabl

error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170926
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170927
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170928
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170929
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20170930
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20171001
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20171002
error in variable depth_of_the_bottom_of_the_euphotic_layer : 530 Sorry, you may not connect more than ten times. 20171003
error in variabl

...

error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170828
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170829
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170830
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170831
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170901
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170902
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170903
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170904
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170905
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20170906


error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171118
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171119
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171120
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171121
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171122
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171123
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171124
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171125
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171126
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20171127


error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180208
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180209
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180210
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180211
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180212
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180213
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180214
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180215
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180216
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180217


error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180501
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180502
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180503
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180504
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180505
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180506
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180507
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180508
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180509
error in variable heated_layer_depth : 530 Sorry, you may not connect more than ten times. 20180510


ValueError: day is out of range for month

Volver a correr estas o revisar: 

"depth_of_the_bottom_of_the_euphotic_layer":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZEU_DAY_00",

"secchi_disk_depth":"__GLOB_RESOLUTION_AVW-MODVIR_ZSD-DORON_DAY",

"heated_layer_depth":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_ZHL_DAY_00",

"inorganic_suspended_particulate_matter_concentration":"__GLOB_RESOLUTION_AVW-MODVIROLAVJ1OLB_SPM-OC5_DAY_00"

    